In [4]:
import open3d as o3d
import numpy as np

# Load a point cloud with vertex colors (replace 'your_point_cloud.ply' with your file)
print('start')
pcd = o3d.io.read_point_cloud("/home/gharib-ml/Desktop/cloud.ply")
# simplify
pcd = pcd.voxel_down_sample(voxel_size=0.1)

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
print(pcd)
# # Visualize the point cloud
o3d.visualization.draw_geometries([pcd])
# print('point cloud loaded')
# # Estimate normals (required for surface reconstruction)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
vertices_to_remove = densities < np.quantile(densities, 0.1)
mesh.remove_vertices_by_mask(vertices_to_remove)
print(mesh)
# radii = [0.01, 0.1, 0.2, 0.4]  # Adjust these values according to the scale of your building

# # Perform surface reconstruction
# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd, o3d.utility.DoubleVector(radii))
o3d.visualization.draw_geometries([mesh])
# print('normals estimated')
# # Optionally, you can orient the normals to be consistent
# pcd.orient_normals_consistent_tangent_plane(100)
# print('normals oriented')
# # Ball Pivoting Algorithm (BPA) for surface reconstruction
# radii = [1.0, 2.0, 4.0]  # Define the radii for the BPA
# mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     pcd, o3d.utility.DoubleVector(radii))
# print('mesh created')
# o3d.visualization.draw_geometries([mesh])
# # Transfer the colors from the point cloud to the mesh
# # Compute vertex normals for the mesh (required for visualization)
# mesh.compute_vertex_normals()
# print('vertex normals computed')
# # Ensure the mesh has vertex colors
# if not any(mesh.has_vertex_colors()):
#     # Assign the vertex colors from the point cloud to the mesh
#     vertex_colors = np.asarray(pcd.colors)
#     mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)

# # Visualize the reconstructed mesh with vertex colors
# o3d.visualization.draw_geometries([mesh])

# # Save the reconstructed mesh
# o3d.io.write_triangle_mesh("reconstructed_mesh_with_colors.ply", mesh)


start
PointCloud with 345793 points.
[Open3D DEBUG] Input Points / Samples: 345793 / 272914
[Open3D DEBUG] #   Got kernel density: 0.09315800666809082 (s), 788.96875 (MB) / 888.125 (MB) / 4279 (MB)
[Open3D DEBUG] #     Got normal field: 0.4918959140777588 (s), 788.96875 (MB) / 888.125 (MB) / 4279 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 4.425733e-06 / 1.530387e+00
[Open3D DEBUG] #       Finalized tree: 0.513085126876831 (s), 830.73046875 (MB) / 888.125 (MB) / 4279 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.4473910331726074 (s), 789.015625 (MB) / 888.125 (MB) / 4279 (MB)
[Open3D DEBUG] #Set point constraints: 0.19373798370361328 (s), 789.015625 (MB) / 888.125 (MB) / 4279 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 1902825 / 2169144 / 5513
[Open3D DEBUG] Memory Usage: 789.016 MB
[Open3D DEBUG] # Linear system solved: 1.3279271125793457 (s), 842.08203125 (MB) / 888.125 (MB) / 4279 (MB)
[Open3D DEBUG] Got average: 0.04173898696899414 (s), 842.08203125 (MB) / 8

In [16]:
print(o3d.__version__)

print(mesh)
mesh.compute_vertex_normals()
mesh.remove_duplicated_vertices()
# Check for non-manifold edges
non_manifold_edges = mesh.get_non_manifold_edges()
if len(non_manifold_edges) > 0:
    print(f"Found {len(non_manifold_edges)} non-manifold edges.")
    # Optional: Attempt to remove non-manifold edges
    mesh.remove_non_manifold_edges()

# Check for non-manifold vertices
non_manifold_vertices = mesh.get_non_manifold_vertices()
if len(non_manifold_vertices) > 0:
    print(f"Found {len(non_manifold_vertices)} non-manifold vertices.")
    # Optional: Attempt to remove non-manifold vertices
    # mesh.remove_non_manifold_vertices()


t_mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)

# Compute the UV atlas partition
try:
    uv_output = t_mesh.compute_uvatlas()
    print("UV unwrapping succeeded.")
except RuntimeError as e:
    print(f"Error during UV unwrapping: {e}")

0.18.0
TriangleMesh with 27590 points and 54547 triangles.
Found 18 non-manifold vertices.
[Open3D INFO] actual parallel_partitions 1
Error during UV unwrapping: [Open3D Error] (void open3d::t::geometry::kernel::uvunwrapping::{anonymous}::ComputeUVAtlasPartition(open3d::t::geometry::TriangleMesh, float, bool, open3d::t::geometry::kernel::uvunwrapping::{anonymous}::UVAtlasPartitionOutput&)) /root/Open3D/cpp/open3d/t/geometry/kernel/UVUnwrapping.cpp:124: UVAtlasPartition: Non-manifold mesh



In [1]:
import pymeshlab
import numpy as np
import open3d as o3d
pcd = o3d.io.read_point_cloud("/home/gharib-ml/Desktop/master_thesis_v2/exps/exp2-1/cloud.ply")
# simplify
pcd = pcd.voxel_down_sample(voxel_size=0.2)

pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.05, max_nn=30))
print(pcd)
# Load point cloud from pcd
# pcd = o3d.io.read_point_cloud("your_point_cloud.ply")
points = np.asarray(pcd.points)
normals = np.asarray(pcd.normals)
colors = np.asarray(pcd.colors)
print('colors',colors[:10,:])
# add alpha channel
colors = np.concatenate([colors, np.ones((colors.shape[0], 1))], axis=1)
print(points.shape, normals.shape, colors.shape)
m = pymeshlab.Mesh(points, v_normals_matrix=normals, v_color_matrix=colors) # color is N x 4 with alpha info
ms = pymeshlab.MeshSet()
ms.add_mesh(m, "pc_scan")
print('point cloud loaded')
ms.generate_surface_reconstruction_screened_poisson(depth=8, scale=1)
# not familiar with the crop API, but I'm sure it's doable
# now we generate UV map; there are a couple options here but this is a naive way
print('surface reconstructed')
ms.compute_texcoord_parametrization_triangle_trivial_per_wedge()
# create texture using UV map and vertex colors
print('uv map computed')
ms.compute_texmap_from_color(textname=f"my_texture_name") # textname will be filename of a png, should not be a full path
# texture file won't be saved until you save the mesh
ms.save_current_mesh(mesh_path)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
PointCloud with 9547 points.
colors [[0.47058824 0.44313725 0.41176471]
 [0.48453159 0.48932462 0.43877996]
 [0.25003381 0.24246112 0.21541582]
 [0.46666667 0.47843137 0.44313725]
 [0.52352941 0.53529412 0.5       ]
 [0.53333333 0.5372549  0.50588235]
 [0.51176471 0.46078431 0.4254902 ]
 [0.49281046 0.49836601 0.4496732 ]
 [0.48976827 0.48955437 0.44884135]
 [0.49717647 0.49654902 0.45164706]]
(9547, 3) (9547, 3) (9547, 4)
point cloud loaded


In [18]:
print(mesh.vertex_colors)

std::vector<Eigen::Vector3d> with 132830 elements.
Use numpy.asarray() to access data.


print()

In [5]:
import pymeshlab

# Load the .ply file
ms = pymeshlab.MeshSet()
ms.load_new_mesh("reconstructed_mesh_with_colors.ply")
ms.compute_texcoord_parametrization_triangle_trivial_per_wedge()
ms.compute_texmap_from_color(textname=f"my_texture_name") # textname will be filename of a png, should not be a full path
# Save the mesh as .dae
ms.save_current_mesh("f.dae", save_textures=True)

Similar Triangles face sampling
